In [3]:
import yfinance as yf
import backtrader as bt
import backtrader.feeds as btfeeds
from datetime import datetime

In [4]:
# Download historical data
data_GLD = yf.download('GLD', start='2011-05-05', end=datetime.now().strftime('%Y-%m-%d'))
data_SPLV = yf.download('SPLV', start='2011-05-05', end=datetime.now().strftime('%Y-%m-%d'))
data_spy = btfeeds.YahooFinanceData(dataname='SPY', fromdate=datetime(2011,5,5), todate=datetime(2022,12,31))
data_agg = btfeeds.YahooFinanceData(dataname='AGG', fromdate=datetime(2011,5,5), todate=datetime(2022,12,31))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
print(data_GLD.index.min())

2011-05-05 00:00:00


In [6]:
print(data_SPLV.index.min())

2011-05-05 00:00:00


In [7]:
class SimpleGold(bt.Strategy):

    def __init__(self):
        self.GLD = self.datas[0]
        self.SPLV = self.datas[1]
        
    def next(self):
        date = self.datas[0].datetime.date(0)
        if ((date.month == 12 and date.day >= 20) or (date.month == 1) or 
            (date.month == 2 and date.day <= 20) or (date.month == 8)):
            if self.getposition(self.SPLV).size:
                self.sell(data=self.SPLV)
            if not self.getposition(self.GLD).size:
                self.buy(data=self.GLD)
        else:
            if self.getposition(self.GLD).size:
                self.sell(data=self.GLD)
            if not self.getposition(self.SPLV).size:
                self.buy(data=self.SPLV)

In [8]:
# Define Buy and Hold Strategy
class BuyAndHold(bt.Strategy):
    def next(self):
        if not self.position:
            self.buy()

In [9]:
# Run Backtest for Buy and Hold
cerebro = bt.Cerebro()
cerebro.addstrategy(BuyAndHold)
cerebro.adddata(data_spy)
cerebro.run()

FileNotFoundError: [Errno 2] No such file or directory: 'SPY'

In [ ]:
# Define 60/40 Strategy
class SixtyForty(bt.Strategy):
    def __init__(self):
        self.spy = self.getdatabyname("SPY")
        self.agg = self.getdatabyname("AGG")

    def next(self):
        # Rebalance the portfolio at the start of each year
        if self.datas[0].datetime.date(0).month == 1 and self.datas[0].datetime.date(0).day == 1:
            self.order_target_percent(self.spy, target=0.6)
            self.order_target_percent(self.agg, target=0.4)

In [ ]:
# Run Backtest for 60/40
cerebro = bt.Cerebro()
cerebro.addstrategy(SixtyForty)
cerebro.adddata(data_spy, name="SPY")
cerebro.adddata(data_agg, name="AGG")
cerebro.run()

In [ ]:
# Create a cerebro
cerebro = bt.Cerebro()

# Add data feeds to cerebro
data1 = bt.feeds.PandasData(dataname=data_GLD)
data2 = bt.feeds.PandasData(dataname=data_SPLV)
cerebro.adddata(data1)
cerebro.adddata(data2)

# Add strategy to cerebro
cerebro.addstrategy(SimpleGold)

# Run the strategy
results = cerebro.run()

# Plot the result
cerebro.plot()

In [ ]:
# Add analyzers
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

results = cerebro.run()
strat = results[0]

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Sharpe Ratio:', strat.analyzers.sharpe.get_analysis())
print('Drawdown:', strat.analyzers.drawdown.get_analysis())